In [1]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
# 1. Path to SQLite with EMA50 / EMA200 already computed
db_path = Path("data") / "processed" / "data_processed.sqlite"


# 2. Load table
with sqlite3.connect(db_path) as conn:
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
    if tables.empty:
        raise RuntimeError(f"No tables found in {db_path}")
    table_name = tables["name"].iloc[0]
    df = pd.read_sql(f'SELECT * FROM "{table_name}"', conn, parse_dates=["Date"])


df = df.sort_values("Date").set_index("Date")


# 3. Ensure required columns exist
for col in ["Close", "EMA50", "EMA200"]:
    if col not in df.columns:
        raise KeyError(f"Required column '{col}' not found; columns = {df.columns.tolist()}")


# 4. EMA50 > EMA200 strategy: signal = 1 if EMA50 > EMA200, else -1
df["signalEMA"] = np.where(df["EMA50"] > df["EMA200"], 1, -1)


# 5. Returns based on signal
df["market_ret"]   = df["Close"].pct_change().fillna(0)
df["strategy_ret"] = df["signalEMA"].shift(1).fillna(0) * df["market_ret"]
df["strategy_eq"]  = (1 + df["strategy_ret"]).cumprod()


print(df[["Close", "EMA50", "EMA200", "signalEMA"]].tail())


# 6. Save EMA signals to SQLite
out_db_path = Path("data") / "signals" / "EMAsignal.sqlite"
out_db_path.parent.mkdir(parents=True, exist_ok=True)


with sqlite3.connect(out_db_path) as conn:
    df.reset_index().to_sql("signals_ema", conn, if_exists="replace", index=False)


print("Saved EMA signals to:", out_db_path.resolve())

                 Close       EMA50      EMA200  signalEMA
Date                                                     
2025-12-01  283.100006  262.094490  236.087363          1
2025-12-02  286.190002  263.039412  236.585897          1
2025-12-03  284.149994  263.867278  237.059172          1
2025-12-04  280.700012  264.527385  237.493409          1
2025-12-05  278.779999  265.086311  237.904221          1
Saved EMA signals to: C:\Users\soltv\Documents\GitHub\newalgotrade\data\signals\EMAsignal.sqlite
